**IMdB sentiment Analysis**

In [1]:
import tensorflow as tf
import numpy as np


In [2]:
import tensorflow_datasets as tfds

In [3]:
raw_train_set, raw_validation_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete47IY4A/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete47IY4A/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete47IY4A/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
tf.random.set_seed(42)

train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)

validation_set = raw_validation_set.batch(32).prefetch(1)

test_set = raw_test_set.batch(32).prefetch(1)

In [8]:
for review, label in raw_train_set.take(2):
  print(review.numpy().decode("utf-8"))
  print("label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

##model

In [9]:
vocab_size = 1000 # lets consider 1000 for now, for unique words that may appear

text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)

text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))



In [10]:
embed_size =128

tf.random.set_seed(42)

sentiment_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim= 128),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

sentiment_model.compile(
    loss= "binary_crossentropy",
    optimizer= "adam",
    metrics=["accuracy"]
)

model_chkpt = tf.keras.callbacks.ModelCheckpoint("IMdB_sentiment_analysis", monitor="val_accuracy")

history = sentiment_model.fit(train_set, validation_data=validation_set, epochs=2, callbacks=[model_chkpt])


Epoch 1/2
704/704 [==============================] - ETA: 0s - loss: 0.6934 - accuracy: 0.5029

704/704 [==============================] - 829s 1s/step - loss: 0.6934 - accuracy: 0.5029 - val_loss: 0.6931 - val_accuracy: 0.5012
Epoch 2/2
704/704 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5048

704/704 [==============================] - 829s 1s/step - loss: 0.6929 - accuracy: 0.5048 - val_loss: 0.6935 - val_accuracy: 0.4972


In [14]:
y_proba = sentiment_model.predict(["The movie was not interesting and boring"])

1/1 [==============================] - 1s 557ms/step


In [15]:
y_proba

array([[0.58249265]], dtype=float32)